Q1. Install the package

In [1]:
!mlflow --version

mlflow, version 2.3.2


Q2. Download and preprocess the data

In [28]:
!python preprocess_data.py --raw_data_path ./TAXI_DATA_FOLDER/ --dest_path ./output

In [29]:
!cd output && dir

 O volume na unidade C n�o tem nome.
 O N�mero de S�rie do Volume � D48C-AA0A

 Pasta de c:\Users\thais.guapyassu\Desktop\DataCourse\Homework2\output

19/05/2023  09:36    <DIR>          .
19/05/2023  09:36    <DIR>          ..
19/05/2023  11:15           153.660 dv.pkl
19/05/2023  11:15         2.632.817 test.pkl
19/05/2023  11:15         2.146.163 train.pkl
19/05/2023  11:15         2.336.393 val.pkl
               4 arquivo(s)      7.269.033 bytes
               2 pasta(s)   368.405.344.256 bytes dispon�veis


Q3. Train a model with autolog

In [4]:
!mlflow ui --backend-store-uri=sqlite:///mlflow.db

^C


In [30]:
!python train.py

2023/05/19 11:16:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\thais.guapyassu\Desktop\DataCourse\venv_name\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [37]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids='1'
)


In [38]:
runs[0].data.params["max_depth"]

'10'

Launch the tracking server locally for MLflow

In [39]:
!mlflow server --backend-store-uri sqlite:///backend.db

^C


In [2]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


Q4. Tune model hyperparameters

In [ ]:
!py hpo.py

In [5]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "sqlite:///backend.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

all_experiments = [exp.experiment_id for exp in mlflow.search_experiments()]

runs = mlflow.search_runs(
    experiment_ids=all_experiments,
    run_view_type=ViewType.ALL,
)

In [6]:
min(runs["metrics.rmse"])

2.449827329704216

Q5. Promote the best model to the model registry

In [ ]:
!py register_model.py

In [24]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///backend.db"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)


for version in latest_versions:
    print(f"run_id: {version.run_id} version: {version.version}, stage: {version.current_stage}")

run_id: 618d2cb55db94585b4498a2459c8f2a6 version: 1, stage: None


In [27]:
from pprint import pprint

client = MlflowClient()
for rm in client.search_registered_models("name='nyc-taxi-regressor'"):
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1685492099300,
    'description': '',
    'last_updated_timestamp': 1685492099412,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1685492099412, current_stage='None', description='', last_updated_timestamp=1685492099412, name='nyc-taxi-regressor', run_id='618d2cb55db94585b4498a2459c8f2a6', run_link='', source='mlflow-artifacts:/2/618d2cb55db94585b4498a2459c8f2a6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>],
    'name': 'nyc-taxi-regressor',
    'tags': {}}


test_rmse	2.285